# **Stage-1:- Creating a Knowledge Base**

- **Problem Statement**
  - Contract / defense data is stored in Excel but it’s hard to search intelligently
  - Traditional keyword search fails when the query wording is different (synonyms / rephrasing)
  - Finding the most relevant past contract descriptions takes too much manual effort
  - Even if you find a match, getting the full context (supplier, program, amount, etc.) from the correct row is difficult
  - A scalable system is needed to support quick retrieval + future AI extraction workflows

- **Proposed Solution**
  - Build a Vector Knowledge Base (KB) from the Excel dataset
  - Convert contract descriptions into semantic embeddings using a transformer model
  - Store embeddings in a FAISS vector index for fast similarity-based retrieval
  - Store all original Excel columns as metadata to return complete structured information
  - Enable searching based on meaning, not only exact words

- **Outcome**
  * Your Excel knowledge base will become **searchable by meaning (semantic search)** instead of only keywords
  * When you give a **new contract description/query**, the system will return the **most similar past contracts** instantly
  * You will be able to retrieve the **best-matching row** even if the words are different (synonyms, rephrasing, short forms)
  * Along with the match, you will also get the **complete row details** (Supplier, Program, Amount, Dates, etc.) because metadata is stored
  * Your extraction pipeline will become **more accurate**, since the LLM can be grounded with relevant historical examples
  * It will reduce **manual lookup time**, improve consistency, and make the process scalable as data grows
  * You will have a reusable **system KB (FAISS + metadata files)** that can be loaded anytime without rebuilding every time
  * This becomes a strong base for building an **agentic workflow** like: Retrieve → Validate → Extract → Store → Report




In [28]:
import os
import re
import pickle
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

DEFAULT_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"


def clean_text(text: str) -> str:
    if text is None:
        return ""
    text = str(text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


def safe_to_str(x):
    if pd.isna(x):
        return ""
    return str(x).strip()


def build_system_kb_store_all_columns(
    excel_path: str,
    save_dir: str = "system_kb_store",
    model_name: str = DEFAULT_MODEL_NAME,
    batch_size: int = 64,
    embed_column: str = "Description of Contract",
):
    os.makedirs(save_dir, exist_ok=True)

    print(f"\n Loading Excel Knowledge Base: {excel_path}")
    df = pd.read_excel(excel_path)
    print(f" Loaded rows={len(df)} cols={len(df.columns)}")

    if embed_column not in df.columns:
        raise ValueError(f"Embed column '{embed_column}' not found in Excel!")

    df = df.fillna("")

    kb_texts = []
    kb_meta = []

    for idx, row in df.iterrows():
        desc = clean_text(row[embed_column])

        # Skip weak text
        if not desc or len(desc) < 20:
            continue

        meta = {"row_id": int(idx)}
        for col in df.columns:
            meta[col] = safe_to_str(row[col])

        meta[embed_column] = desc

        kb_texts.append(desc)
        kb_meta.append(meta)

    print(f"KB rows kept after cleaning: {len(kb_texts)}")

    if len(kb_texts) == 0:
        print("ERROR: No text rows remained after cleaning.")
        return None, None

    print(f"Loading embedding model: {model_name}")
    embedder = SentenceTransformer(model_name)

    print("Creating embeddings...")
    embeddings = []

    for i in range(0, len(kb_texts), batch_size):
        batch = kb_texts[i : i + batch_size]
        batch_emb = embedder.encode(
            batch, show_progress_bar=True, normalize_embeddings=True
        )
        embeddings.append(batch_emb)

    embeddings = np.vstack(embeddings).astype("float32")
    dim = embeddings.shape[1]

    print(f"Embedding shape: {embeddings.shape}")

    # Cosine similarity via Inner Product (since normalized)
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)

    index_path = os.path.join(save_dir, "system_kb.faiss")
    meta_path = os.path.join(save_dir, "system_kb_meta.pkl")

    faiss.write_index(index, index_path)
    with open(meta_path, "wb") as f:
        pickle.dump(kb_meta, f)

    print("\n System KB Created Successfully!")
    print(f"Index saved: {index_path}")
    print(f"Meta saved : {meta_path}")

    return index_path, meta_path

# Part 2: Retriever

class SystemKBRetriever:
    def __init__(self, kb_dir="system_kb_store", model_name=DEFAULT_MODEL_NAME):
        index_path = os.path.join(kb_dir, "system_kb.faiss")
        meta_path = os.path.join(kb_dir, "system_kb_meta.pkl")

        if not os.path.exists(index_path) or not os.path.exists(meta_path):
            raise FileNotFoundError("❌ KB files missing. Build KB first.")

        print(f"\n Loading FAISS index: {index_path}")
        self.index = faiss.read_index(index_path)

        print(f"Loading metadata: {meta_path}")
        with open(meta_path, "rb") as f:
            self.meta = pickle.load(f)

        print(f"Loaded KB rows: {len(self.meta)}")
        print(f"Loading embedder: {model_name}")
        self.embedder = SentenceTransformer(model_name)

    def retrieve(self, query_text: str, top_k: int = 5):
        query_text = str(query_text).strip()
        if not query_text:
            return []

        q_emb = self.embedder.encode([query_text], normalize_embeddings=True).astype(
            "float32"
        )

        scores, idxs = self.index.search(q_emb, top_k)
        results = []

        for score, idx in zip(scores[0], idxs[0]):
            if idx < 0:
                continue

            results.append({"score": float(score), "meta": self.meta[idx]})

        return results

# Run Full Pipeline (Build + Search)

if __name__ == "__main__":
    EXCEL_PATH = r"C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\data\sample_data.xlsx"
    KB_DIR = "system_kb_store"

    # Step 1: Build KB
    build_system_kb_store_all_columns(
        excel_path=EXCEL_PATH,
        save_dir=KB_DIR,
        model_name=DEFAULT_MODEL_NAME,
        batch_size=64,
        embed_column="Description of Contract",
    )

    # Step 2: Load KB + Retrieve
    r = SystemKBRetriever(kb_dir=KB_DIR)

    query = "Dell Marketing L.P., Round Rock, Texas, is awarded a single-award, firm-fixed-price blanket purchase agreement"

    hits = r.retrieve(query, top_k=3)

    print("\n" + "=" * 60)
    print("TOP MATCHES")
    print("=" * 60)

    for i, h in enumerate(hits, start=1):
        print(f"\n🔹 Rank: {i}")
        print("Score:", h["score"])
        print("Supplier:", h["meta"].get("Supplier Name", ""))
        print("Market:", h["meta"].get("Market Segment", ""))
        print("System:", h["meta"].get("System Name (Specific)", ""))
        print("Row ID:", h["meta"].get("row_id", ""))


c:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



 Loading Excel Knowledge Base: C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\data\sample_data.xlsx
 Loaded rows=2068 cols=29
KB rows kept after cleaning: 600
Loading embedding model: sentence-transformers/all-MiniLM-L6-v2
Creating embeddings...


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.06s/it]


Embedding shape: (600, 384)

 System KB Created Successfully!
Index saved: system_kb_store\system_kb.faiss
Meta saved : system_kb_store\system_kb_meta.pkl

 Loading FAISS index: system_kb_store\system_kb.faiss
Loading metadata: system_kb_store\system_kb_meta.pkl
Loaded KB rows: 600
Loading embedder: sentence-transformers/all-MiniLM-L6-v2

TOP MATCHES

🔹 Rank: 1
Score: 0.6228203773498535
Supplier: Dell Inc
Market: Unknown
System: Department of Defense Enterprise Software Initiative (DOD ESI)
Row ID: 379

🔹 Rank: 2
Score: 0.6228203773498535
Supplier: Dell Inc
Market: Unknown
System: Department of Defense Enterprise Software Initiative (DOD ESI)
Row ID: 1

🔹 Rank: 3
Score: 0.6024371385574341
Supplier: 
Market: Unknown
System: Department of Defense Enterprise Software Initiative (DOD ESI)
Row ID: 159


## **Stage 2**

In this stage I will be creating AI agest that can help in extraction of data based on the input. 

In [29]:
import os, sys
import re
import json
import pickle
import difflib
import datetime
from typing import Annotated, TypedDict, List, Dict, Any, Optional, Tuple

import pandas as pd
import faiss
from dateutil import parser
from dateutil.relativedelta import relativedelta
import getpass

# LangGraph / LangChain
from langchain_core.messages import AnyMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from openai import OpenAI

# Excel formatting
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font

sys.path.append(os.getcwd())

from prompts import (
    GEOGRAPHY_PROMPT, 
    SYSTEM_CLASSIFIER_PROMPT, 
    CONTRACT_EXTRACTOR_PROMPT, 
    VALIDATOR_FIX_PROMPT
)

**Configurations and Supporting File Path**

In [30]:
## LLM CLIENT SETUP (OpenRouter)

if "LLMFOUNDRY_TOKEN" not in os.environ:
    os.environ["LLMFOUNDRY_TOKEN"] = getpass.getpass("Enter the LLM Foundry API Key: ")

client = OpenAI(
    api_key=f'{os.environ.get("LLMFOUNDRY_TOKEN")}:my-test-project',
    base_url="https://llmfoundry.straive.com/openai/v1/",
)


OPENROUTER_MODEL = "gpt-4o-mini"

# Token control per stage
MAXTOK_STAGE2 = 250
MAXTOK_STAGE3 = 850
MAXTOK_STAGE4 = 350
MAXTOK_STAGE7 = 350

# CONFIGURATION & FILE PATHS
TAXONOMY_PATH = r"C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\notebook\taxonomy.json"
SUPPLIERS_PATH = r"C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\notebook\suppliers.json"
INPUT_EXCEL_PATH = r"C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\data\source_file.xlsx"
OUTPUT_CSV_PATH = "Processed_Defense_Data.csv"
RAG_KB_DIR = r"C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\system_kb_store"

**Helper Functions for Stage-2**

In [31]:
## RULE BOOK + GEOGRAPHY

RULE_BOOK = {
    "defensive_countermeasures": {
        "triggers": ["flare", "chaff", "countermeasure", "decoy", "mju-", "ale-"],
        "guidance": "Market Segment: 'C4ISR Systems', System Type (General): 'Defensive Systems', Specific: 'Defensive Aid Suite'"
    },
    "radars_and_sensors": {
        "triggers": ["radar", "sonar", "sensor", "an/apy", "an/tpy"],
        "guidance": "Market Segment: 'C4ISR Systems', System Type (General): 'Sensors'"
    },
    "ammunition": {
        "triggers": ["cartridge", "round", "projectile", " 5.56", " 7.62", "ammo"],
        "guidance": "Market Segment: 'Weapon Systems', System Type (General): 'Ammunition'"
    }
}

GEOGRAPHY_MAPPING = {
    "Sub-Saharan Africa": [
        "Angola", "Benin", "Botswana", "Burkina Faso", "Burundi", "Cameroon", "Cape Verde",
        "Central African Republic", "Chad", "Congo, Democratic Republic of", "Congo, Republic of",
        "Djibouti", "Equatorial Guinea", "Eritrea", "Eswatini", "Ethiopia", "Gabon", "Gambia",
        "Ghana", "Guinea", "Guinea-Bissau", "Ivory Coast", "Kenya", "Lesotho", "Liberia",
        "Madagascar", "Malawi", "Mali", "Mauritius", "Mozambique", "Namibia", "Niger",
        "Nigeria", "Rwanda", "Senegal", "Seychelles", "Sierra Leone", "Somalia", "South Africa",
        "South Sudan", "Sudan", "Tanzania", "Togo", "Uganda", "Zambia", "Zimbabwe"
    ],
    "Asia-Pacific": [
        "Australia", "Brunei", "Cambodia", "China", "Hong Kong", "Indonesia", "Japan", "Laos",
        "Malaysia", "Mongolia", "Myanmar", "New Zealand", "North Korea", "Papua New Guinea",
        "Philippines", "Singapore", "South Korea", "Taiwan", "Thailand", "Vietnam"
    ],
    "Europe": [
        "Albania", "Austria", "Belgium", "Bosnia and Herzegovina", "Bulgaria", "Croatia", "Cyprus",
        "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Georgia", "Germany", "Greece",
        "Hungary", "Iceland", "Ireland", "Italy", "Kosovo", "Latvia", "Lithuania", "Luxembourg",
        "Malta", "Montenegro", "Netherlands", "North Macedonia", "Norway", "Poland", "Portugal",
        "Romania", "Serbia", "Slovakia", "Slovenia", "Spain", "Sweden", "Switzerland", "Turkey",
        "Ukraine", "United Kingdom"
    ],
    "Latin America": [
        "Argentina", "Bahamas", "Barbados", "Belize", "Bolivia", "Brazil", "Chile", "Colombia",
        "Costa Rica", "Cuba", "Curacao", "Dominican Republic", "Ecuador", "El Salvador", "Guatemala",
        "Guyana", "Haiti", "Honduras", "Jamaica", "Mexico", "Nicaragua", "Panama", "Paraguay",
        "Peru", "Suriname", "Trinidad and Tobago", "Uruguay", "Venezuela"
    ],
    "Middle East and North Africa": [
        "Algeria", "Bahrain", "Egypt", "Iran", "Iraq", "Israel", "Jordan", "Kuwait", "Lebanon",
        "Libya", "Mauritania", "Morocco", "Oman", "Qatar", "Saudi Arabia", "Syria", "Tunisia",
        "United Arab Emirates", "Yemen"
    ],
    "North America": ["Canada", "USA"],
    "Russia & CIS": [
        "Armenia", "Azerbaijan", "Belarus", "Kazakhstan", "Kyrgyzstan", "Moldova", "Russia",
        "Tajikistan", "Turkmenistan", "Uzbekistan"
    ],
    "South Asia": [
        "Afghanistan", "Bangladesh", "India", "Maldives", "Nepal", "Pakistan", "Sri Lanka"
    ],
    "Unknown": [
        "Andorra", "Antigua and Barbuda", "Bhutan", "Comoros", "Dominica", "Federated States of Micronesia",
        "Fiji", "Grenada", "Kiribati", "Liechtenstein", "Marshall Islands", "Monaco", "Nauru", "Palau",
        "Palestine", "Puerto Rico", "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and the Grenadines",
        "Samoa", "San Marino", "Sao Tom and Principe", "Solomon Islands", "Timor-Leste", "Tonga", "Tuvalu",
        "Unknown", "Vanuatu", "Vatican City", "Western Sahara"
    ]
}

ALLOWED_OPERATORS = [
    "Army",
    "Navy",
    "Air Force",
    "Defence Wide",
    "Ukraine (Assistance)",
    "Foreign Assistance",
    "Other"
]

PROGRAM_TYPE_ALLOWED = [
    "Procurement",
    "Training",
    "MRO/Support",
    "RDT&E",
    "Upgrade",
    "Other Service"
]

DESIGNATOR_PATTERNS = [
    r"\bDDG[-\s]?\d+\b", r"\bCVN[-\s]?\d+\b", r"\bSSN[-\s]?\d+\b",
    r"\bLCS[-\s]?\d+\b", r"\bLPD[-\s]?\d+\b", r"\bLHA[-\s]?\d+\b", r"\bLHD[-\s]?\d+\b",
    r"\bF-\d+\b", r"\bB-\d+\b", r"\bC-\d+\b", r"\bA-\d+\b",
    r"\bMQ-\d+\b", r"\bRQ-\d+\b",
    r"\bAN\/[A-Z0-9\-]+\b",
    r"\b(AIM|AGM|SM|RIM|MIM)-\d+\b",
]

In [32]:
def chunk_text(text: str, chunk_size: int = 1800, overlap: int = 250) -> List[str]:
    """
    Deterministic chunking: stable + safe + prevents token overflow.
    - chunk_size: number of characters per chunk
    - overlap: overlapping characters to preserve boundaries
    """
    text = str(text)
    if not text.strip():
        return []

    chunks = []
    start = 0
    n = len(text)

    while start < n:
        end = min(start + chunk_size, n)
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)

        if end >= n:
            break

        start = max(0, end - overlap)

    return chunks
#--------------------------------------------

def pick_best_non_empty(values: List[str]) -> str:
    """Return first strong value else empty."""
    for v in values:
        if v and str(v).strip() and str(v).strip().lower() not in ["unknown", "n/a", "not applicable", "none"]:
            return str(v).strip()
    return ""
#--------------------------------------------

def normalize_customer_operator(raw_text: str) -> str:
    """
    Strictly maps input text to the allowed SOP drop-down list.
    """
    if not raw_text:
        return "Unknown"
        
    text = str(raw_text).strip().lower()

    # 1. Ukraine (Assistance) - High Priority
    # Detects: Ukraine Security Assistance Initiative, USAI, or explicitly "for Ukraine"
    if "ukraine" in text:
        return "Ukraine (Assistance)"

    # 2. Foreign Assistance
    # Detects: FMS, Foreign Military Sales, Foreign Customers
    if any(k in text for k in ["foreign military sales", "fms", "foreign customer", "foreign government"]):
        return "Foreign Assistance"

    # 3. Air Force (Includes Space Force per SOP)
    if any(k in text for k in ["space force", "ussf", "air force", "usaf"]):
        return "Air Force"

    # 4. Navy (Includes Marine Corps as they are Dept of Navy, unless 'Other' is preferred)
    # Note: SOP didn't explicitly list Marine Corps. Standard practice maps USMC to Navy. 
    # If strict separation is needed, move 'marine' to Other. 
    if any(k in text for k in ["navy", "usn", "marine", "usmc", "naval"]):
        return "Navy"

    # 5. Army
    if any(k in text for k in ["army", "usa ", "u.s. army"]): # Space padding to avoid matching "army" inside other words
        return "Army"

    # 6. Defence Wide
    # Detects: DLA, MDA, DTRA, DARPA, DISA, DCMA
    if any(k in text for k in ["defense logistics", "dla", "missile defense", "mda", "defense wide", "defence wide", "darpa", "disa"]):
        return "Defence Wide"

    # 7. Other
    # Detects: Coast Guard (per SOP), DIA, Joint, Special Ops
    if any(k in text for k in ["coast guard", "uscg", "defense intelligence", "dia", "joint", "special operations", "socom"]):
        return "Other"

    # Fallback to Other if it looks like a government agency but didn't match above
    if len(text) > 2:
        return "Other"

    return "Unknown"

#--------------------------------------------

def load_json_file(filename, default_value):
    try:
        with open(filename, "r", encoding="utf-8") as f:
            print(f"Loaded: {filename}")
            return json.load(f)
    except Exception as e:
        print(f"Warning: Could not load {filename} ({e}). Using default.")
        return default_value
#--------------------------------------------

def normalize_country_name(c: str) -> str:
    """
    Standardizes country names to ensure consistent Region mapping.
    """
    if not c:
        return "Unknown"
    
    t = str(c).strip().lower()
    
    # Common variations map
    mappings = {
        "usa": "USA", "us": "USA", "u.s.": "USA", "united states": "USA", 
        "united states of america": "USA", "america": "USA",
        "uk": "United Kingdom", "u.k.": "United Kingdom", "britain": "United Kingdom", 
        "great britain": "United Kingdom", "england": "United Kingdom",
        "uae": "United Arab Emirates",
        "sk": "South Korea", "rok": "South Korea", "republic of korea": "South Korea",
        "prc": "China", "people's republic of china": "China"
    }

    if t in mappings:
        return mappings[t]
        
    # Return capitalized version if no match found
    return str(c).strip().title()
#--------------------------------------------

def normalize_program_type(pt: str) -> str:
    if not pt:
        return "Other Service"

    t = str(pt).strip().lower()

    if any(k in t for k in ["mro", "support", "maintenance", "repair", "overhaul", "sustainment", "logistics"]):
        return "MRO/Support"
    if "training" in t:
        return "Training"
    if any(k in t for k in ["rdte", "research", "development", "prototype", "test and evaluation"]):
        return "RDT&E"
    if any(k in t for k in ["upgrade", "modernization", "modification"]):
        return "Upgrade"
    if any(k in t for k in ["procure", "buy", "purchase", "production", "delivery"]):
        return "Procurement"

    return "Other Service"
#--------------------------------------------

raw_taxonomy = load_json_file(TAXONOMY_PATH, {})
TAXONOMY_STR = json.dumps(raw_taxonomy, separators=(",", ":"))

SUPPLIER_LIST = load_json_file(SUPPLIERS_PATH, [
    "Dell Inc", "Boeing", "Lockheed Martin", "Raytheon Technologies",
    "Northrop Grumman", "L3Harris", "BAE Systems", "General Dynamics"
])

#--------------------------------------------
def get_best_supplier_match(extracted_name: str) -> str:
    if not extracted_name or str(extracted_name).strip().lower() in ["unknown", "n/a", "not applicable"]:
        return "Unknown"

    raw_name = str(extracted_name).strip()
    raw_lower = raw_name.lower()
    valid_suppliers = sorted([str(s) for s in SUPPLIER_LIST], key=len, reverse=True)

    # 1) Exact match
    for s in valid_suppliers:
        if s.lower() == raw_lower:
            return s

    # 2) Known inside extracted
    for s in valid_suppliers:
        if len(s) > 3 and s.lower() in raw_lower:
            return s

    # 3) Extracted inside known
    for s in valid_suppliers:
        if raw_lower in s.lower():
            return s

    # 4) Strict fuzzy
    matches = difflib.get_close_matches(raw_name, valid_suppliers, n=1, cutoff=0.8)
    if matches:
        return matches[0]

    return raw_name

#--------------------------------------------

def extract_awardee_supplier_strict(paragraph: str) -> Tuple[str, str]:
    text = str(paragraph).strip()

    patterns = [
        r"^([A-Z][A-Za-z0-9&\-\.\s]+?),\s+.*?\s+(?:is|was|has been)\s+awarded\b",
        r"^([A-Z][A-Za-z0-9&\-\.\s]+?),\s+.*?\s+received\s+an?\s+award\b",
    ]

    for pat in patterns:
        m = re.search(pat, text, flags=re.IGNORECASE)
        if m:
            raw_supplier = m.group(1).strip()
            final_supplier = get_best_supplier_match(raw_supplier)
            return final_supplier, raw_supplier

    return "Unknown", "Not Found"

#--------------------------------------------

def calculate_mro_months(start_date_str, end_date_text, program_type):
    if program_type != "MRO/Support":
        return "Not Applicable"
    try:
        if not start_date_str or not end_date_text:
            return "Not Applicable"

        start = pd.to_datetime(start_date_str, dayfirst=True)
        end = parser.parse(str(end_date_text), fuzzy=True)

        diff = relativedelta(end, start)
        total_months = diff.years * 12 + diff.months
        return str(max(0, int(total_months)))
    except:
        return "Not Applicable"

#--------------------------------------------

def get_region_for_country(country_name):
    if not country_name or str(country_name).strip().lower() in ["unknown", "n/a", "not applicable"]:
        return "Unknown"

    clean = str(country_name).strip().lower()

    if clean in ["us", "usa", "u.s.", "united states", "united states of america"]:
        return "North America"
    if clean in ["uk", "u.k.", "britain", "great britain"]:
        return "Europe"

    for region, countries in GEOGRAPHY_MAPPING.items():
        if any(c.lower() == clean for c in countries):
            return region

    return "Unknown"

def extract_designators(text: str):
    text = str(text)
    found = []
    for pat in DESIGNATOR_PATTERNS:
        found.extend(re.findall(pat, text, flags=re.IGNORECASE))
    cleaned = [f.upper().replace(" ", "").replace("--", "-") for f in found]
    final, seen = [], set()
    for x in cleaned:
        if x not in seen:
            final.append(x)
            seen.add(x)
    return final


def detect_piloting_rule_based(text: str, designators: List[str]) -> str:
    t = str(text).lower()

    if any(d.startswith(("MQ-", "RQ-")) for d in designators):
        return "Uncrewed"
    if any(k in t for k in ["unmanned", "uav", "drone", "autonomous"]):
        return "Uncrewed"

    if any(d.startswith(("DDG", "CVN", "SSN", "LCS", "LPD", "LHA", "LHD")) for d in designators):
        return "Crewed"
    if "uss " in t:
        return "Crewed"

    return "Not Applicable"

def detect_piloting_strict(text: str, designators: List[str], system_type: str = "") -> str:
    """
    Strictly determines System Piloting based on SOP definitions.
    """
    t = str(text).lower()
    
    # 1. OPTIONAL (Explicitly stated)
    if any(k in t for k in ["optionally manned", "optional piloting", "manned-unmanned teaming", "manned/unmanned"]):
        return "Optional"

    # 2. UNCREWED (UAVs, Drones, Autonomous)
    # Check designators first (Strong signal)
    if any(d.startswith(("MQ-", "RQ-", "XQ-", "MQ-", "RQ-")) for d in designators):
        return "Uncrewed"
    # Check keywords
    if any(k in t for k in ["unmanned", "uav", "uas", "drone", "autonomous", "remotely piloted", "rpa"]):
        return "Uncrewed"

    # 3. CREWED (Requires human pilot/driver)
    # Check designators (Ships, Fighters, Bombers, Transports)
    if any(d.startswith(("DDG", "CVN", "SSN", "LCS", "LPD", "LHA", "LHD", "CG-", "FFG")) for d in designators):
        return "Crewed"
    if any(d.startswith(("F-", "B-", "C-", "A-", "AH-", "UH-", "CH-", "MH-")) and not d.startswith("C-UAS") for d in designators):
        return "Crewed"
    # Check keywords
    if any(k in t for k in ["manned", "crew", "pilot", "cockpit", "fighter aircraft", "helicopter", "submarine", "frigate", "destroyer", "carrier"]):
        # Exclusion: "unmanned" check above handles "unmanned surface vessel"
        if "unmanned" not in t: 
            return "Crewed"

    # 4. NOT APPLICABLE (Default for non-vehicles)
    # If the system is clearly a component, weapon, or support item, it's N/A.
    # We use a broad check for "Not Applicable" candidates.
    na_indicators = [
        "missile", "munition", "bomb", "ammunition", "round", "cartridge", 
        "radar", "sensor", "radio", "software", "training", "simulator", 
        "engine", "spare part", "container", "gun", "artillery", "howitzer",
        "maintenance", "support", "logistics", "service"
    ]
    if any(k in t for k in na_indicators):
        return "Not Applicable"
        
    # If specific system type is known from Stage 3 and it's not a platform
    if system_type and system_type not in ["Air Platform", "Naval Platform", "Land Platform"]:
         return "Not Applicable"

    # Default fallback if ambiguous (often best to leave blank or N/A, but SOP says N/A for non-driving)
    return "Not Applicable"


def normalize_program_type_improved(pt_llm: str, text: str) -> str:
    """
    Improved Program Type logic with expanded keywords and better conflict resolution.
    """
    text_lower = str(text).lower()
    pt_llm = str(pt_llm).strip()

    # 1. RDT&E (Strongest signal)
    if any(k in text_lower for k in ["rdt&e", "research", "development", "prototype", "demonstration", "sbir", "sttr", "study", "design phase"]):
        return "RDT&E"

    # 2. MRO/Support (Strong signal)
    # Added: "sustainment", "cls" (contractor logistics support), "pbl" (performance based logistics)
    if any(k in text_lower for k in ["mro", "sustainment", "maintenance", "repair", "overhaul", "logistics", "support services", "cls", "technical support", "engineering support"]):
        return "MRO/Support"

    # 3. Upgrade (Specific action on existing)
    if any(k in text_lower for k in ["upgrade", "modernization", "retrofit", "modification", "life extension", "sle", "update"]):
        return "Upgrade"

    # 4. Procurement (Broadest category, tricky vs Training)
    # Explicit "Procurement" signals
    procure_signals = ["production", "delivery", "procurement", "acquisition", "supply", "purchase", "manufacture", "fabrication", "assembly"]
    if any(k in text_lower for k in procure_signals):
        return "Procurement"

    # Training HARDWARE check (Simulators -> Procurement)
    if "training" in text_lower or "simulator" in text_lower:
        if any(k in text_lower for k in ["simulator", "device", "trainer", "hardware", "system", "kit", "aids"]):
             return "Procurement"
    
    # 5. Training (Services Only)
    # If it says "training" and wasn't caught by Procurement above
    if "training" in text_lower or "instruction" in text_lower:
        return "Training"

    # 6. Other Service (Catch-all for services)
    if "service" in text_lower or "labor" in text_lower:
        return "Other Service"

    # Fallback: Trust LLM if it found a valid category
    valid_types = ["Procurement", "Training", "MRO/Support", "RDT&E", "Upgrade", "Other Service"]
    if pt_llm in valid_types:
        return pt_llm

    # Default if truly unknown but looks like a contract award
    if "award" in text_lower:
        return "Procurement" # Safe default for "awarded contract for X"

    return "Unknown"


def calculate_value_certainty_strict(text: str, value_str: str) -> str:
    """
    Highly recommends 'Confirmed'. Only 'Estimated' if explicitly stated.
    """
    text_lower = text.lower()
    
    # 1. Strong "Confirmed" Default
    # If we extracted a specific non-zero value, assume Confirmed unless proven otherwise.
    if not value_str or value_str == "0.000":
        return "Estimated" # No value = Estimated/Unknown

    # 2. Explicit Estimation Modifiers (Strict)
    # Only tag 'Estimated' if these words explicitly modify the value context.
    # "Ceiling" implies IDIQ max, which is an estimate of potential spend.
    estimation_signals = ["estimated value", "estimated cost", "ceiling", "maximum value", "potential value", "not to exceed"]
    
    if any(k in text_lower for k in estimation_signals):
        return "Confirmed"

    # 3. IDIQ / BPA (Usually Ceiling = Estimated)
    # However, if user wants "Confirmed" for the stated face value, we can be lenient.
    # SOP says: "Select Estimated when the value is not confirmed and possibility of future modifications."
    # IDIQ ceilings ARE limits, not confirmed spend. So we keep them as Estimated.
    if "indefinite-delivery" in text_lower or "idiq" in text_lower:
         # Often IDIQs have a "face value" (initial task order) vs "ceiling".
         # If the text says "awarded a $X task order", it's confirmed.
         # If it says "awarded a $X IDIQ contract", it's a ceiling (Estimated).
         if "task order" in text_lower or "delivery order" in text_lower:
             return "Estimated" 
         return "Confirmed"

    return "Confirmed"

def extract_award_value_million_strict(paragraph: str) -> str:
    """
    Extract contract AWARD value strictly from award patterns only.
    Avoids funding amounts later in the paragraph.
    Output: string in million format like '12.015'
    """
    t = str(paragraph)

    patterns = [
        r"\bis awarded a\s+\$([\d,]+(?:\.\d+)?)\b",
        r"\bare awarded an estimated aggregate ceiling of\s+\$([\d,]+(?:\.\d+)?)\b",
        r"\bare awarded an?\s+\$([\d,]+(?:\.\d+)?)\b",
        r"\bceiling of\s+\$([\d,]+(?:\.\d+)?)\b",
    ]

    for pat in patterns:
        m = re.search(pat, t, flags=re.IGNORECASE)
        if m:
            try:
                val = float(m.group(1).replace(",", ""))
                return f"{val/1_000_000:.3f}"
            except:
                pass

    return ""


def detect_quantity_should_be_na(paragraph: str) -> bool:
    """
    If paragraph contains many item quantities -> treat Quantity as Not Applicable.
    Example3: 483 missiles, 82 missiles, 156 missiles, 198 containers...
    That is NOT a single contract-level quantity.
    """
    t = str(paragraph)

    # Count numeric patterns that look like item quantities
    qty_candidates = re.findall(r"\b(\d{1,5})\b", t)

    # If too many numbers appear, it's almost always a line-item contract
    if len(qty_candidates) >= 12:
        return True

    # Specific strong markers for line-item heavy paragraphs
    if any(k in t.lower() for k in ["as follows:", "lot", "containers", "spare", "tail cap", "guidance unit"]):
        return True

    return False


def normalize_currency(cur: str) -> str:
    if not cur:
        return "USD$"
    c = str(cur).strip().upper()
    if c in ["USD", "US$", "$", "US DOLLAR", "DOLLARS"]:
        return "USD$"
    return c

def word_to_int(token: str) -> Optional[int]:
    """
    Converts word numbers to int:
    eight -> 8, twenty one -> 21

    Supports 0-99 safely.
    Returns None if not a valid word-number.
    """
    if not token:
        return None

    t = str(token).strip().lower()
    t = t.replace("-", " ")
    parts = [p for p in t.split() if p]

    ones = {
        "zero": 0, "one": 1, "two": 2, "three": 3, "four": 4, "five": 5,
        "six": 6, "seven": 7, "eight": 8, "nine": 9, "ten": 10,
        "eleven": 11, "twelve": 12, "thirteen": 13, "fourteen": 14,
        "fifteen": 15, "sixteen": 16, "seventeen": 17, "eighteen": 18,
        "nineteen": 19
    }

    tens = {
        "twenty": 20, "thirty": 30, "forty": 40, "fifty": 50,
        "sixty": 60, "seventy": 70, "eighty": 80, "ninety": 90
    }

    # single word
    if len(parts) == 1:
        if parts[0] in ones:
            return ones[parts[0]]
        if parts[0] in tens:
            return tens[parts[0]]
        return None

    # two words: "twenty one"
    if len(parts) == 2:
        if parts[0] in tens and parts[1] in ones:
            return tens[parts[0]] + ones[parts[1]]
        return None

    return None


def parse_qty_token(qty_token: str) -> Optional[int]:
    """
    Converts numeric token or word-number token into int.
    """
    if qty_token is None:
        return None
    s = str(qty_token).strip().lower()

    # numeric
    if re.fullmatch(r"\d+", s):
        return int(s)

    # word-number
    return word_to_int(s)


def parse_fms_countries(paragraph: str) -> List[str]:
    """
    Extract FMS customer countries list.
    Example: 'governments of Australia, Bahrain, Belgium...'
    """
    text = str(paragraph)

    m = re.search(
        r"governments of (.+?)(?:\.\s| Work will be performed| Fiscal| This contract|$)",
        text,
        flags=re.IGNORECASE | re.DOTALL
    )
    if not m:
        return []

    block = m.group(1)
    raw = re.split(r",|\band\b", block)

    countries = []
    for c in raw:
        c = c.strip()
        if 2 < len(c) <= 40:
            countries.append(c)

    final, seen = [], set()
    for c in countries:
        if c.lower() not in seen:
            final.append(c)
            seen.add(c.lower())

    return final


def detect_multiple_supplier_award(paragraph: str) -> bool:
    """
    Detect Example2 pattern:
    Many suppliers listed with semicolons + "are awarded ..."
    """
    t = str(paragraph).strip()
    if ";" in t and re.search(r"\bare awarded\b", t, flags=re.IGNORECASE):
        if re.search(r"\([A-Z0-9]{6,}\)", t):
            return True
    return False


def parse_line_item_operator_allocations(paragraph: str) -> List[Dict[str, str]]:
    """
    Extract line-item splits like:

    '483 AIM-9X ... missiles (212 for the Navy, 187 for the Air Force and 84 for FMS customers);'
    '82 AIM-9X ... missiles (eight for the Navy, eight for the Air Force and 66 for FMS customers);'

    Output:
      [
        {"item_name": "...", "operator":"Navy", "quantity":"212", "g2g_b2g":"B2G"},
        {"item_name": "...", "operator":"Air Force", "quantity":"187", "g2g_b2g":"B2G"},
        {"item_name": "...", "operator":"Foreign Assistance", "quantity":"84", "g2g_b2g":"G2G"}
      ]
    """
    text = str(paragraph)

    # This finds: <total qty> <item desc> ( <allocation block> )
    # Example: 483 AIM-9X ... missiles (212 for the Navy, 187 for the Air Force and 84 for FMS customers)
    item_pattern = r"\b(\d+)\s+(.+?)\s*\(([^)]*for[^)]*)\)"
    matches = re.findall(item_pattern, text, flags=re.IGNORECASE)

    results = []

    for total_qty, item_desc, alloc_block in matches:
        item_desc_clean = re.sub(r"\s+", " ", item_desc).strip(" ;,. -")

        # --- BRANCH allocations (digit or word-number)
        branch_pattern = r"\b(\d+|[A-Za-z\-]+)\s+for\s+the\s+(Navy|Air Force|Army|Marine Corps|Space Force|Coast Guard|Defense Logistics Agency|Missile Defense Agency)\b"
        for qty_token, op_raw in re.findall(branch_pattern, alloc_block, flags=re.IGNORECASE):
            qty_val = parse_qty_token(qty_token)
            if qty_val is None:
                continue
            normalized_op = normalize_customer_operator(op_raw)       
            results.append({
                "item_name": item_desc_clean,
                "operator": normalized_op,
                "quantity": str(qty_val),
                "g2g_b2g": "B2G"
            })

        # --- FMS allocations (digit or word-number)
        fms_pattern = r"\b(\d+|[A-Za-z\-]+)\s+for\s+(?:Foreign Military Sales\s*\(FMS\)\s*customers|Foreign Military Sales\s*customers|FMS\s*customers|FMS)\b"
        for qty_token in re.findall(fms_pattern, alloc_block, flags=re.IGNORECASE):
            qty_val = parse_qty_token(qty_token)
            if qty_val is None:
                continue

            results.append({
                "item_name": item_desc_clean,
                "operator": "Foreign Assistance",
                "quantity": str(qty_val),
                "g2g_b2g": "G2G"
            })

    return results


def split_rows_engine(base_row: dict, paragraph: str) -> List[dict]:
    """
    FINAL STAGE5 Split Engine (Your required behavior)

    SPLITS:
    1) Multi-supplier award => Supplier Name = Multiple (NO supplier split)
    2) Line-item + operator allocation split (Navy/AirForce/FMS)
    3) FMS country split ONLY for G2G rows

    Only split fields are modified, rest remain unchanged.
    """
    paragraph = str(paragraph).strip()
    row = base_row.copy()

    split_reasons = []

    # ------------------------------------------------------------------
    # 1) MULTIPLE SUPPLIER AWARD (Example2)
    # ------------------------------------------------------------------
    if detect_multiple_supplier_award(paragraph):
        row["Supplier Name"] = "Multiple"
        row["Supplier Name Evidence"] = "Multiple Suppliers (Detected)"
        row["Split Flag"] = "No"
        row["Split Reason"] = "Multiple supplier award detected (no supplier split)"
        return [row]

    rows = [row]

    # ------------------------------------------------------------------
    # 2) LINE-ITEM + OPERATOR SPLIT (Your expected output)
    # ------------------------------------------------------------------
    item_allocs = parse_line_item_operator_allocations(paragraph)

    if item_allocs:
        split_reasons.append("Line-item operator allocation split")

        new_rows = []
        for r in rows:
            for alloc in item_allocs:
                rr = r.copy()

                # Only split fields change
                rr["Customer Operator"] = alloc["operator"]
                rr["Quantity"] = alloc["quantity"]
                rr["G2G/B2G"] = alloc["g2g_b2g"]

                # Make system labels reflect line-item name (matches your expected output)
                rr["System Name (General)"] = alloc["item_name"]
                rr["System Name (Specific)"] = alloc["item_name"]

                new_rows.append(rr)

        rows = new_rows

    # ------------------------------------------------------------------
    # 3) FMS COUNTRY SPLIT (ONLY for G2G rows)
    # ------------------------------------------------------------------
    fms_countries = parse_fms_countries(paragraph)
    if fms_countries:
        split_reasons.append("FMS country split")

        final_rows = []
        for r in rows:
            is_g2g = str(r.get("G2G/B2G", "")).strip().upper() == "G2G"
            is_fms_operator = "foreign" in str(r.get("Customer Operator", "")).lower()

            if is_g2g or is_fms_operator:
                for c in fms_countries:
                    rr = r.copy()
                    rr["Customer Country"] = normalize_country_name(c)
                    rr["Customer Region"] = get_region_for_country(rr["Customer Country"])
                    final_rows.append(rr)
            else:
                final_rows.append(r)

        rows = final_rows

    # ------------------------------------------------------------------
    # Final flags
    # ------------------------------------------------------------------
    if split_reasons:
        reason = " | ".join(split_reasons)
        for r in rows:
            r["Split Flag"] = "Yes"
            r["Split Reason"] = reason
    else:
        for r in rows:
            r["Split Flag"] = "No"
            r["Split Reason"] = "No split condition found"

    return rows

## LLM CALL HELPER (OpenRouter Safe Wrapper)
def call_llm_json(system_prompt: str, user_prompt: str, max_tokens: int):
    """
    Safe OpenRouter call wrapper
    - JSON response enforced
    - max_tokens enforced
    """
    completion = client.chat.completions.create(
        model=OPENROUTER_MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0,
        max_tokens=max_tokens,
        response_format={"type": "json_object"},
    )
    return json.loads(completion.choices[0].message.content)

# 11) EXCEL HIGHLIGHTING FEATURE

def highlight_evidence_reason_columns(excel_path: str):
    wb = load_workbook(excel_path)

    # Always ensure at least one visible sheet
    ws = wb.active
    ws.sheet_state = "visible"

    header = [cell.value for cell in ws[1]]

    evidence_cols = []
    reason_cols = []

    for idx, col_name in enumerate(header, start=1):
        if isinstance(col_name, str) and "Evidence" in col_name:
            evidence_cols.append(idx)
        if isinstance(col_name, str) and "Reason" in col_name:
            reason_cols.append(idx)

    evidence_fill = PatternFill(start_color="FFF2CC", end_color="FFF2CC", fill_type="solid")
    reason_fill = PatternFill(start_color="D9E1F2", end_color="D9E1F2", fill_type="solid")
    header_font = Font(bold=True)

    for col_idx in evidence_cols:
        ws.cell(row=1, column=col_idx).fill = evidence_fill
        ws.cell(row=1, column=col_idx).font = header_font

    for col_idx in reason_cols:
        ws.cell(row=1, column=col_idx).fill = reason_fill
        ws.cell(row=1, column=col_idx).font = header_font

    for row in range(2, ws.max_row + 1):
        for col_idx in evidence_cols:
            ws.cell(row=row, column=col_idx).fill = evidence_fill
        for col_idx in reason_cols:
            ws.cell(row=row, column=col_idx).fill = reason_fill

    wb.save(excel_path)
    print("Evidence + Reason columns highlighted successfully.")


Loaded: C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\notebook\taxonomy.json
Loaded: C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\notebook\suppliers.json


**Using Knowledgebase to support my extraction**

In [33]:
## RAG RETRIEVER (FAISS + METADATA)
class SystemKBRetriever:
    def __init__(self, kb_dir: str, embed_model: str = "sentence-transformers/all-MiniLM-L6-v2"):
        self.kb_dir = kb_dir
        self.embed_model = embed_model

        index_path = os.path.join(kb_dir, "system_kb.faiss")
        meta_path = os.path.join(kb_dir, "system_kb_meta.pkl")

        if not os.path.exists(index_path) or not os.path.exists(meta_path):
            raise FileNotFoundError(
                f"KB files not found in: {kb_dir}\n"
                f"Expected:\n- {index_path}\n- {meta_path}\n\n"
                f"Build KB first using your KB builder script."
            )
        print(f"Loading FAISS Index: {index_path}")
        self.index = faiss.read_index(index_path)

        print(f"Loading KB Metadata: {meta_path}")
        with open(meta_path, "rb") as f:
            self.meta = pickle.load(f)

        print(f"KB Loaded rows: {len(self.meta)}")

        self.embedder = None

    def _lazy_load_embedder(self):
        if self.embedder is None:
            from sentence_transformers import SentenceTransformer
            self.embedder = SentenceTransformer(self.embed_model)

    def retrieve(self, query_text: str, top_k: int = 3):
        import numpy as np
        query_text = str(query_text).strip()
        if not query_text:
            return []

        self._lazy_load_embedder()

        q_emb = self.embedder.encode([query_text], normalize_embeddings=True).astype("float32")
        scores, idxs = self.index.search(q_emb, top_k)

        results = []
        for score, idx in zip(scores[0], idxs[0]):
            if idx < 0:
                continue
            results.append({"score": float(score), "meta": self.meta[idx]})
        return results

In [34]:
retriever = SystemKBRetriever(kb_dir=RAG_KB_DIR)

Loading FAISS Index: C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\system_kb_store\system_kb.faiss
Loading KB Metadata: C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\system_kb_store\system_kb_meta.pkl
KB Loaded rows: 600


In [35]:
def safe_str(x, default=""):
    """
    Safely converts input to string, returning default if None or empty/whitespace.
    """
    if x is None:
        return default
    s = str(x).strip()
    return s if s else default

def rag_best_hit(paragraph: str, top_k: int = 3):
    """
    Retrieves the single best match from the Knowledge Base (FAISS).
    Returns: (hit_object, score, metadata_dict)
    """
    # Ensure the global retriever is loaded
    if 'retriever' not in globals():
        print("Warning: 'retriever' is not defined. Returning empty hit.")
        return None, 0.0, {}

    hits = retriever.retrieve(paragraph, top_k=top_k)
    
    if not hits:
        return None, 0.0, {}
        
    best = hits[0]
    return best, float(best.get("score", 0.0)), best.get("meta", {})

#-------------------------------------------
RAG_STRONG_THRESHOLD = 0.78   # if >= this, trust KB fully
RAG_MEDIUM_THRESHOLD = 0.70   # if >= this, use KB as strong hint

def normalize_country_name(c: str) -> str:
    if not c:
        return "Unknown"
    t = str(c).strip().lower()
    if t in ["united states", "united states of america", "us", "u.s.", "usa", "america"]:
        return "USA"
    if t in ["united kingdom", "uk", "u.k.", "britain", "great britain"]:
        return "UK"
    return str(c).strip()

def rag_best_hit(paragraph: str, top_k: int = 3):
    hits = retriever.retrieve(paragraph, top_k=top_k)
    if not hits:
        return None, 0.0, {}
    best = hits[0]
    return best, float(best.get("score", 0.0)), best.get("meta", {})

def safe_str(x, default=""):
    if x is None:
        return default
    s = str(x).strip()
    return s if s else default

def normalize_program_type_strict(pt_llm: str, text: str) -> str:
    """
    Decides Program Type based on strict SOP hierarchy.
    Priority: RDT&E > MRO > Upgrade > Procurement (Hardware/Simulators) > Training (Services).
    """
    text_lower = str(text).lower()
    pt_llm = str(pt_llm).strip()

    # 1. RDT&E (Highest Priority)
    # Detects: Prototypes, Research, Development, SBIR
    if any(k in text_lower for k in ["rdt&e", "research", "development", "prototype", "sbir", "demonstration"]):
        return "RDT&E"

    # 2. MRO/Support
    # Detects: Sustainment, Maintenance, Logistics, Repair, Overhaul
    if any(k in text_lower for k in ["mro", "sustainment", "maintenance", "logistics", "repair", "overhaul", "support services"]):
        return "MRO/Support"

    # 3. Upgrade
    # Detects: Modernization, Retrofit, Modification of existing platforms
    if any(k in text_lower for k in ["upgrade", "modernization", "retrofit", "modification", "life extension"]):
        return "Upgrade"

    # 4. Training (Strict Split)
    # Rule: Training SERVICES = Training.
    # Rule: Training HARDWARE (Simulators, Aircraft) = Procurement.
    if "training" in text_lower or "training" in pt_llm.lower():
        # Check for hardware indicators
        if any(k in text_lower for k in ["simulator", "device", "trainer aircraft", "hardware", "system", "kit"]):
            return "Procurement"
        return "Training"

    # 5. Procurement
    # Detects: Production, Delivery, Acquisition, Construction
    if any(k in text_lower for k in ["production", "delivery", "procurement", "acquisition", "supply", "purchase"]):
        return "Procurement"

    # Fallback to LLM's choice if valid, otherwise Other
    if pt_llm in ["Procurement", "Training", "MRO/Support", "RDT&E", "Upgrade", "Other Service"]:
        return pt_llm

    return "Other Service"


def calculate_value_certainty(text: str, value_str: str) -> str:
    """
    Determines if value is 'Confirmed' or 'Estimated'.
    """
    text_lower = text.lower()
    
    # 1. Keywords indicating estimation/limit
    if any(k in text_lower for k in ["ceiling", "maximum", "estimated", "potential", "not to exceed", "idiq", "bpa", "blanket purchase agreement"]):
        return "Estimated"

    # 2. Shared/Split Value Indicators
    # If multiple companies share a pot, the value is Estimated for that specific row
    if any(k in text_lower for k in ["shared", "competing", "each awarded", "multiple award", "aggregate"]):
        return "Estimated"

    # 3. If no value was extracted, it can't be confirmed
    if not value_str or value_str == "0.000":
        return "Estimated"

    return "Confirmed"


def calculate_usd_value(val_million_str: str, currency: str) -> str:
    """
    Populates 'Value (USD$ Million)'.
    Assuming 1:1 if already USD, otherwise leaves blank or requires conversion logic.
    """
    try:
        val = float(val_million_str)
        if val == 0:
            return ""
            
        # If Currency is USD, copy the value
        if "USD" in currency.upper() or "$" in currency:
            return f"{val:.3f}"
            
        # (Optional) Add simplistic conversion if needed, e.g., GBP -> 1.25 * Val
        # For now, we return extraction.
        return f"{val:.3f}"
    except:
        return ""

## AGENTS / TOOLS (Chunk-wise extraction + Merge)

In [36]:
## AGENTS / TOOLS (Chunk-wise extraction + Merge)

# Stage 1: SOURCING EXTRACTOR

class SourcingInput(BaseModel):
    paragraph: str = Field(description="Full contract paragraph/description text.")
    url: str = Field(description="Source URL of the contract announcement/news.")
    date: str = Field(description="Contract date in Excel (string).")

@tool("sourcing_extractor")
def sourcing_extractor(paragraph: str, url: str, date: str):
    """
    Stage 1: SOURCING EXTRACTOR

    Purpose:
    - Creates the base skeleton row (stable fields).

    Output columns created:
    - Description of Contract
    - Additional Notes (Internal Only)
    - Source Link(s)
    - Contract Date
    - Reported Date (By SGA)

    Important:
    - These fields remain SAME even after splits.
    - Every split row inherits these values.
    """
    reported_date = datetime.datetime.now().strftime("%Y-%m-%d")

    notes = "Standard extraction."
    if "modification" in str(paragraph).lower():
        notes = "Contract Modification."
    if "multiple award" in str(paragraph).lower():
        notes = "Multiple award contract detected (non-supplier split)."

    return {
        "Description of Contract": paragraph,
        "Additional Notes (Internal Only)": notes,
        "Source Link(s)": url,
        "Contract Date": date,
        "Reported Date (By SGA)": reported_date
    }


In [37]:
## # Stage 2: GEOGRAPHY EXTRACTOR (Chunk -> Merge)

class GeographyInput(BaseModel):
    paragraph: str = Field(description="Full contract paragraph/description text.")

@tool("geography_extractor")
def geography_extractor(paragraph: str):
    """
    Stage 2: GEOGRAPHY EXTRACTOR
    Updates Supplier Region based on the extracted Supplier Country.
    """
    paragraph = str(paragraph).strip()
    if not paragraph:
        return {}

    # --- RAG Optimization ---
    best_hit, best_score, best_meta = rag_best_hit(paragraph, top_k=3)
    
    # Defaults
    cust_country = "Unknown"
    cust_op = "Unknown"
    supp_country = "Unknown"

    if best_hit and best_score >= 0.78:
        # Trust KB if high confidence
        cust_country = safe_str(best_meta.get("Customer Country", ""))
        cust_op = normalize_customer_operator(safe_str(best_meta.get("Customer Operator", "")))
        supp_country = safe_str(best_meta.get("Supplier Country", ""))
    else:
        # LLM Extraction
        chunks = chunk_text(paragraph, chunk_size=1800, overlap=250)
        outputs = []
        for ch in chunks:
            try:
                raw = call_llm_json(GEOGRAPHY_PROMPT, ch, max_tokens=250)
                outputs.append(raw)
            except Exception:
                continue
        
        cust_country = pick_best_non_empty([o.get("Customer Country") for o in outputs]) or "Unknown"
        cust_op = normalize_customer_operator(pick_best_non_empty([o.get("Customer Operator") for o in outputs]))
        supp_country = pick_best_non_empty([o.get("Supplier Country") for o in outputs]) or "Unknown"

    # Normalize Countries
    cust_country = normalize_country_name(cust_country)
    supp_country = normalize_country_name(supp_country)

    # Calculate Regions using the SHARED logic
    cust_region = get_region_for_country(cust_country)
    supp_region = get_region_for_country(supp_country) # Same list as Customer Region

    return {
        "Customer Region": cust_region,
        "Customer Country": cust_country,
        "Customer Operator": cust_op,
        "Supplier Region": supp_region,   # Extracted here
        "Supplier Country": supp_country,
        "Domestic Content": "Indigenous" if cust_country == supp_country else "Imported"
    }


In [38]:
# Stage 3: SYSTEM CLASSIFIER (Chunk -> Merge)

class SystemInput(BaseModel):
    paragraph: str = Field(description="Full contract paragraph/description text.")
    item_focus: str = Field(description="Specific line-item focus (from Stage5), e.g. 'All up round containers'")

@tool("system_classifier")
def system_classifier(paragraph: str, item_focus: str = ""):
    """
    Stage 3: SYSTEM CLASSIFIER (Updated with Strict Piloting)
    """
    paragraph = str(paragraph).strip()
    
    # RAG & LLM Logic (Same as before)
    rag_query = f"{item_focus} {paragraph}" if item_focus else paragraph
    rag_hits = retriever.retrieve(rag_query, top_k=3)
    rag_context = json.dumps([h['meta'] for h in rag_hits if h.get('score', 0) > 0.6], indent=2)
    
    rule_hint = "No specific overrides."
    if "container" in item_focus.lower(): rule_hint = "Classify as Support Equipment / Containers."

    formatted_system_prompt = SYSTEM_CLASSIFIER_PROMPT.format(
        taxonomy_reference=TAXONOMY_STR,
        rule_book_overrides=rule_hint
    )

    user_prompt = f"""
    ITEM_FOCUS: {item_focus if item_focus else "Main System"}
    TEXT: {paragraph}
    SIMILAR RAG EXAMPLES: {rag_context}
    """

    try:
        res = call_llm_json(formatted_system_prompt, user_prompt, max_tokens=850)
        
        # Apply Improved Piloting Logic
        designators = extract_designators(paragraph)
        # Pass the extracted system type to help decide "Not Applicable"
        sys_type = res.get("System Type (General)", "")
        res["System Piloting"] = detect_piloting_strict(paragraph, designators, sys_type)
        
        return res
    except Exception as e:
        return {"Confidence": "Low", "Error": str(e)}

In [ ]:
# Stage 4: CONTRACT EXTRACTOR (Chunk -> Merge + Strict Supplier)

class ContractInfoInput(BaseModel):
    paragraph: str = Field(description="Full contract paragraph/description text.")
    contract_date: str = Field(description="Contract date as string.")

@tool("contract_extractor")
def contract_extractor(paragraph: str, contract_date: str):
    """
    Stage 4: CONTRACT EXTRACTOR (Updated to use LLM for Value Extraction)
    """
    paragraph = str(paragraph).strip()

    # 1. Regex Supplier (Keep this, regex is good for Proper Nouns)
    supplier_name, supplier_evidence = extract_awardee_supplier_strict(paragraph)

    # 2. Define a specialized prompt for Value extraction that "understands" context
    # We ask the LLM to identify the AWARD value specifically, distinct from obligations.
    value_extraction_prompt = """
    You are a defense contract analyst. Extract the following from the text:
    1. "value_million": The total face value of the contract action in MILLIONS (float).
       - Ignore "obligated" amounts or "fiscal funds" unless they are the ONLY value present.
       - If the text says "modification... in the amount of $X", use X.
       - If the text says "ceiling of $Y", use Y.
    2. "currency": The currency code (default USD).
    3. "program_type": Classify as Procurement, Training, MRO/Support, RDT&E, Upgrade, or Other.
    4. "g2g_b2g": "G2G" if Foreign Military Sales (FMS), otherwise "B2G".

    Return JSON only.
    """

    user_prompt = f"CONTRACT TEXT: {paragraph}\nDATE: {contract_date}"

    try:
        # Re-using your existing call_llm_json helper
        llm_data = call_llm_json(value_extraction_prompt, user_prompt, max_tokens=350)
    except Exception as e:
        print(f"LLM Error: {e}")
        llm_data = {}

    # 3. Extract Value from LLM response (Fallback to 0.000)
    val_float = llm_data.get("value_million", 0.0)
    # Ensure it's a string with 3 decimal places for consistency
    try:
        val_formatted = f"{float(val_float):.3f}"
    except:
        val_formatted = "0.000"

    # 4. Apply Logic (Program Type, Currency, etc.)
    llm_pt = llm_data.get("program_type", "")
    final_program_type = normalize_program_type_improved(llm_pt, paragraph)
    
    # Use your strict certainty logic, but now applied to the LLM-found value
    final_certainty = calculate_value_certainty_strict(paragraph, val_formatted)
    currency = normalize_currency(llm_data.get("currency", "USD"))
    usd_val_million = calculate_usd_value(val_formatted, currency)
    mro_months = calculate_mro_months(contract_date, llm_data.get("completion_date_text", ""), final_program_type)

    return {
        "Supplier Name": supplier_name,
        "Supplier Name Evidence": supplier_evidence,
        "Program Type": final_program_type,
        "Value Certainty": final_certainty,
        "Value (Million)": val_formatted,      # Now populated by LLM understanding
        "Value (USD$ Million)": usd_val_million,
        "Currency": currency,
        "Expected MRO Contract Duration (Months)": mro_months,
        "Quantity": "Not Applicable",
        "G2G/B2G": llm_data.get("g2g_b2g", "B2G"),
        "Value Note (If Any)": llm_data.get("value_note", "")
    }

In [40]:
# Stage 5: SPLITTER AGENT (FULL PARAGRAPH ALWAYS)

class SplitterInput(BaseModel):
    paragraph: str = Field(description="Full contract paragraph/description text.")
    base_row: dict = Field(description="Extracted row after Stage1-4.")

@tool("splitter_agent")
def splitter_agent(paragraph: str, base_row: dict):
  """
  Stage 5: SPLITTER AGENT
  Purpose:
    - Applies deterministic split logic to generate multiple output rows
      when paragraph has explicit multi allocations.
  Supported splits:
    - Operator/Quantity split ("212 for the Navy", "187 for the Air Force")
    - FMS country split (only for rows marked as G2G)
    - Multi-value note (does not split, only notes)

  IMPORTANT:
    - Supplier split is REMOVED to prevent wrong supplier explosions.
  """
  try:
      rows = split_rows_engine(base_row, paragraph)
      for r in rows:
          r.setdefault("Split Flag", "No")
          r.setdefault("Split Reason", "")
      return {"rows": rows}
  except Exception as e:
      base_row["Split Flag"] = "Error"
      base_row["Split Reason"] = f"Split failed: {str(e)}"
      return {"rows": [base_row]}


In [41]:
# Stage 6: QUALITY VALIDATOR AGENT

class QAInput(BaseModel):
    paragraph: str = Field(description="Original paragraph for reference.")
    rows: list = Field(description="Final split rows list output from Stage5.")

@tool("quality_validator")
def quality_validator(paragraph: str, rows: list):
    """
    Stage 6: QUALITY VALIDATOR (Hybrid: Rule-Based + KB Check)
    """
    text = str(paragraph).lower()
    validated_rows = []

    for r in rows:
        flags = []
        fixes = []

        # --- Rule Based Checks ---
        supplier = str(r.get("Supplier Name", "")).strip()
        if supplier.lower() in ["unknown", "", "multiple"]:
             # If text clearly has an award, flag it
             if "awarded" in text:
                 flags.append("Supplier Unknown")
                 fixes.append("Check LLM Fallback")

        # --- KB/Taxonomy Validation (NEW) ---
        sys_name = str(r.get("System Name (General)", "")).strip()
        
        # If we have a system name, verify it against KB context
        if sys_name and sys_name.lower() != "unknown":
            # Quick check: does this system appear in our RAG hits?
            # We assume 'retriever' is globally available as per notebook scope
            hits = retriever.retrieve(sys_name, top_k=1)
            if hits:
                top_score = hits[0]['score']
                # If score is very low, the system might be hallucinated or poorly named
                if top_score < 0.35: 
                    flags.append(f"System Name '{sys_name}' has low KB confidence ({top_score:.2f})")
                    fixes.append("Verify system name against standard taxonomy")

        # --- Final Status ---
        qa_status = "PASS" if not flags else "FAIL"
        
        rr = r.copy()
        rr["QA Status"] = qa_status
        rr["QA Flags"] = " | ".join(flags) if flags else "None"
        rr["QA Fix Suggestion"] = " | ".join(fixes) if fixes else "None"
        validated_rows.append(rr)

    return {"rows": validated_rows}

In [42]:
# Stage 7: LLM VALIDATOR (FAIL ONLY) - Chunk Aware
class LLMValidateInput(BaseModel):
    paragraph: str = Field(description="Original paragraph text.")
    row: dict = Field(description="One FAIL row to validate/correct.")

@tool("llm_fail_row_validator")
def llm_fail_row_validator(paragraph: str, row: dict):
    """
    Stage 7: VALIDATOR FIX (Linked to prompts.py)
    """
    # LINK THE PROMPT
    formatted_fix_prompt = VALIDATOR_FIX_PROMPT.format(
        failed_row_json=json.dumps(row, indent=2),
        program_type_enum=str(PROGRAM_TYPE_ALLOWED)
    )

    user_prompt = f"ORIGINAL TEXT CHUNK: {paragraph[:2000]}"

    try:
        fix = call_llm_json(formatted_fix_prompt, user_prompt, max_tokens=350)
        
        # Merge Fixes
        corrected = row.copy()
        if fix.get("Supplier Name") and fix["Supplier Name"] != "Unknown":
            corrected["Supplier Name"] = fix["Supplier Name"]
        if fix.get("Program Type"):
            corrected["Program Type"] = fix["Program Type"]
        if fix.get("Value (Million)"):
            corrected["Value (Million)"] = fix["Value (Million)"]
            
        corrected["LLM QA Fix Summary"] = fix.get("Fix Summary", "Auto-fixed by Agent")
        return {"row": corrected}

    except Exception:
        return {"row": row}

In [43]:
class AgentState(TypedDict):
    input_text: str
    input_date: str
    input_url: str

    final_data: dict
    final_rows: list
    validated_rows: list
    final_rows_post_llm: list

    messages: Annotated[List[AnyMessage], add_messages]


def stage_1_sourcing(state: AgentState):
    res = sourcing_extractor.invoke({
        "paragraph": state["input_text"],
        "url": state["input_url"],
        "date": state["input_date"],
    })
    new_data = state.get("final_data", {}).copy()
    new_data.update(res)
    return {"final_data": new_data}


def stage_2_geography(state: AgentState):
    res = geography_extractor.invoke({"paragraph": state["input_text"]})
    new_data = state.get("final_data", {}).copy()
    new_data.update(res)
    return {"final_data": new_data}


def stage_3_system(state: AgentState):
    """
    NEW Stage3 Node:
    Runs system classification PER split row using item_focus.
    """

    paragraph = state["input_text"]
    rows = state.get("final_rows", [])

    updated_rows = []

    for r in rows:
        item_focus = str(r.get("System Name (Specific)", "")).strip()

        res = system_classifier.invoke({
            "paragraph": paragraph,
            "item_focus": item_focus
        })

        rr = r.copy()
        rr.update(res)   # merge system labels into the row
        updated_rows.append(rr)

    return {"final_rows": updated_rows}


def stage_4_contract(state: AgentState):
    res = contract_extractor.invoke({
        "paragraph": state["input_text"],
        "contract_date": state["input_date"]
    })
    new_data = state.get("final_data", {}).copy()
    new_data.update(res)
    return {"final_data": new_data}


def stage_5_split(state: AgentState):
    """
    Stage5 Node: SplitterEngine

     Ensures output is ALWAYS stored in `final_rows`
    so Stage3 SystemClassifierRAG can loop through them.
    """

    base_row = state.get("final_data", {}) or {}
    paragraph = state.get("input_text", "")

    try:
        res = splitter_agent.invoke({
            "paragraph": paragraph,
            "base_row": base_row
        })

        rows = res.get("rows", None)

        # Hard safety fallback
        if not rows or not isinstance(rows, list):
            rows = [base_row]

        return {"final_rows": rows}

    except Exception as e:
        # Never crash pipeline due to split failure
        fallback = base_row.copy()
        fallback["Split Flag"] = "Error"
        fallback["Split Reason"] = f"SplitterEngine failed: {str(e)}"
        return {"final_rows": [fallback]}


def stage_6_quality_validator(state: AgentState):
    res = quality_validator.invoke({
        "paragraph": state["input_text"],
        "rows": state["final_rows"]
    })
    return {"validated_rows": res.get("rows", state["final_rows"])}


def stage_7_llm_fix_fail_rows(state: AgentState):
    paragraph = state["input_text"]
    validated_rows = state.get("validated_rows", [])

    fixed_rows = []
    for r in validated_rows:
        if r.get("QA Status") == "FAIL":
            fix_res = llm_fail_row_validator.invoke({"paragraph": paragraph, "row": r})
            fixed_rows.append(fix_res.get("row", r))
        else:
            fixed_rows.append(r)

    return {"final_rows_post_llm": fixed_rows}

def node_system_classifier_rag(state: AgentState):
    """
    Stage3 Node (AFTER SplitterEngine)

    Runs system classification per split-row using:
       item_focus = row["System Name (Specific)"] or row["System Name (General)"]

    Updates each row with Market Segment / System Type / System Name / Evidence / Reason
    """

    paragraph = state["input_text"]
    rows = state.get("final_rows", [])

    # If split engine didn't create rows, fallback to single final_data row
    if not rows:
        base = state.get("final_data", {})
        rows = [base] if base else []

    updated_rows = []

    for r in rows:
        item_focus = str(r.get("System Name (Specific)", "")).strip()
        if not item_focus:
            item_focus = str(r.get("System Name (General)", "")).strip()

        # invoke your Stage3 tool
        sys_res = system_classifier.invoke({
            "paragraph": paragraph,
            "item_focus": item_focus
        })

        rr = r.copy()
        rr.update(sys_res)
        updated_rows.append(rr)

    return {"final_rows": updated_rows}


In [44]:
workflow = StateGraph(AgentState)

workflow.add_node("SourcingExtractor", stage_1_sourcing)
workflow.add_node("GeographyExtractor", stage_2_geography)
workflow.add_node("ContractExtractor", stage_4_contract)
workflow.add_node("SplitterEngine", stage_5_split)
workflow.add_node("SystemClassifierRAG", node_system_classifier_rag)  # 
workflow.add_node("QualityValidator", stage_6_quality_validator)
workflow.add_node("LLMFailRowFixer", stage_7_llm_fix_fail_rows)

workflow.add_edge(START, "SourcingExtractor")
workflow.add_edge("SourcingExtractor", "GeographyExtractor")
workflow.add_edge("GeographyExtractor", "ContractExtractor")
workflow.add_edge("ContractExtractor", "SplitterEngine")   # 
workflow.add_edge("SplitterEngine", "SystemClassifierRAG") # 
workflow.add_edge("SystemClassifierRAG", "QualityValidator")
workflow.add_edge("QualityValidator", "LLMFailRowFixer")
workflow.add_edge("LLMFailRowFixer", END)
app = workflow.compile()



app = workflow.compile()

In [45]:
from IPython.display import Image, display
try:
  display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
  print(e)

Failed to reach https://mermaid.ink API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`


In [46]:
# 10) GRAPH EXPORT (OFFLINE SAFE)

def export_workflow_mermaid(app_obj, out_file="workflow.mmd"):
    mmd = app_obj.get_graph().draw_mermaid()
    with open(out_file, "w", encoding="utf-8") as f:
        f.write(mmd)
    print(f"Workflow Mermaid saved locally: {out_file}")
    return out_file

In [47]:
if __name__ == "__main__":

    print(f"\n Loading Input File: {INPUT_EXCEL_PATH}")
    
    # Define Output Path as Excel
    OUTPUT_EXCEL_PATH = "Processed_Defense_Data.xlsx"

    export_workflow_mermaid(app, out_file="workflow.mmd")

    try:
        df_input = pd.read_excel(INPUT_EXCEL_PATH)
        
        # Basic validation
        required_cols = ["Source URL", "Contract Date", "Contract Description"]
        # Allow loose matching or strip whitespace from columns if needed
        df_input.columns = [c.strip() for c in df_input.columns]
        
        if not all(col in df_input.columns for col in required_cols):
             raise ValueError(f"Excel file must contain columns: {required_cols}")

        print(f"Processing {len(df_input)} rows...")
        results = []

        for index, row in df_input.iterrows():
            print(f"\n Row {index + 1}/{len(df_input)}")

            desc = str(row["Contract Description"]) if pd.notna(row["Contract Description"]) else ""
            c_date = str(row["Contract Date"]) if pd.notna(row["Contract Date"]) else str(datetime.date.today())
            c_url = str(row["Source URL"]) if pd.notna(row["Source URL"]) else ""

            initial_state: AgentState = {
                "input_text": desc,
                "input_date": c_date,
                "input_url": c_url,
                "final_data": {},
                "final_rows": [],
                "validated_rows": [],
                "final_rows_post_llm": [],
                "messages": []
            }

            output_state = app.invoke(initial_state)

            # Hierarchy of fallback for getting rows
            rows = output_state.get("final_rows_post_llm", [])
            if not rows:
                rows = output_state.get("validated_rows", [])
            if not rows:
                rows = output_state.get("final_rows", [])
            if not rows:
                rows = [output_state.get("final_data", {})]

            results.extend(rows)

        df_final = pd.DataFrame(results)

        FINAL_COLUMNS = [
            "Customer Region", "Customer Country", "Customer Operator",
            "Supplier Region", "Supplier Country", "Domestic Content",

            "Split Flag", "Split Reason",

            "Market Segment", "Market Segment Evidence", "Market Segment Reason",
            "System Type (General)", "System Type (General) Evidence", "System Type (General) Reason",
            "System Type (Specific)", "System Type (Specific) Evidence", "System Type (Specific) Reason",
            "System Name (General)", "System Name (General) Evidence", "System Name (General) Reason",
            "System Name (Specific)", "System Name (Specific) Evidence", "System Name (Specific) Reason",
            "System Piloting", "System Piloting Evidence", "System Piloting Reason",
            "Confidence",

            "Supplier Name", "Supplier Name Evidence",
            "Program Type", "Expected MRO Contract Duration (Months)",
            "Quantity", "Value Certainty", "Value (Million)", "Currency",
            "Value (USD$ Million)", "Value Note (If Any)", "G2G/B2G",
            "Signing Month", "Signing Year",

            "QA Status", "QA Flags", "QA Fix Suggestion",
            "LLM QA Fix Summary",

            "Description of Contract",
            "Additional Notes (Internal Only)",
            "Source Link(s)",
            "Contract Date",
            "Reported Date (By SGA)"
        ]

        # Reindex ensures all columns exist, filling missing ones with empty string
        df_final = df_final.reindex(columns=FINAL_COLUMNS, fill_value="")

        # 1. Save as Excel
        print(f"Saving to Excel: {OUTPUT_EXCEL_PATH}")
        df_final.to_excel(OUTPUT_EXCEL_PATH, index=False, engine='openpyxl')
        
        # 2. Apply Formatting/Highlighting
        print("Applying Highlighting...")
        highlight_evidence_reason_columns(OUTPUT_EXCEL_PATH)

        print("\nProcessing Complete!")
        print(df_final[["Supplier Name", "Value (USD$ Million)", "Customer Operator", "Supplier Region"]].head(3).to_string(index=False))

    except Exception as e:
        print(f"\n ERROR: {e}")
        import traceback
        traceback.print_exc()


 Loading Input File: C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\data\source_file.xlsx
Workflow Mermaid saved locally: workflow.mmd
Processing 30 rows...

 Row 1/30


C:\Users\mukeshkr\AppData\Local\Temp\ipykernel_12420\604486570.py:212: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  start = pd.to_datetime(start_date_str, dayfirst=True)



 Row 2/30

 Row 3/30

 Row 4/30

 Row 5/30

 Row 6/30

 Row 7/30

 Row 8/30

 Row 9/30

 Row 10/30

 Row 11/30

 Row 12/30

 Row 13/30

 Row 14/30

 Row 15/30

 Row 16/30

 Row 17/30

 Row 18/30

 Row 19/30

 Row 20/30

 Row 21/30

 Row 22/30

 Row 23/30

 Row 24/30

 Row 25/30

 Row 26/30

 Row 27/30

 Row 28/30

 Row 29/30

 Row 30/30
Saving to Excel: Processed_Defense_Data.xlsx
Applying Highlighting...
Evidence + Reason columns highlighted successfully.

Processing Complete!
                    Supplier Name Value (USD$ Million) Customer Operator Supplier Region
BAE Systems - Norfolk Ship Repair              107.736              Navy   North America
                           Boeing               25.648              Navy   North America
                         Intermat               17.736              Navy   North America
